# Introduction

# Imports

In [ ]:
import os
import matplotlib.pyplot as plt

from skimage.exposure import rescale_intensity
import numpy as np
import cv2
from skimage import filters

# Data Access and Loading

In [ ]:
# Need to get Google Drive access
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
img_dir = os.path.join('/content/gdrive/My Drive/2020-tata-memorial-workshop')

# Image Display and Visualization

In [ ]:
image = plt.imread(os.path.join(img_dir, '8865_500_f00003_original.tif'))

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8,16))

axes[0].imshow(image)
axes[0].set_title('Original Image')

axes[1].imshow(image[:250, :250, :])
axes[1].set_title('Original Image (Crop)')

for ax in axes:
  ax.axis('off')
plt.show()

# Basic Image Analysis: Nuclei Detection

## Color Deconvolution for Pathology Stain Separation

See the [scikit-image gallery example](https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_ihc_color_separation.html#sphx-glr-auto-examples-color-exposure-plot-ihc-color-separation-py) as well as [documentation for stain separation](https://scikit-image.org/docs/stable/api/skimage.color.html#skimage.color.separate_stains) for a list of the different convolutional matrices you can import.

In [ ]:
from skimage.color import separate_stains, hed_from_rgb

img_separated = separate_stains(image, hed_from_rgb)
img_hema = img_separated[:,:,0]
img_eosin = img_separated[:,:,1]
img_dab = img_separated[:,:,2]

In [ ]:
# Display
fig, ax = plt.subplots(2, 4, figsize=(20,10))

# Original
ax[0][0].imshow(image)
ax[0][0].set_title('Original')
ax[0][1].imshow(img_hema, cmap=plt.cm.gray)
ax[0][1].set_title('Hematoxylin')
ax[0][2].imshow(img_eosin, cmap=plt.cm.gray)
ax[0][2].set_title('Eosin')
ax[0][3].imshow(img_dab, cmap=plt.cm.gray)
ax[0][3].set_title('DAB')

# Crop
ax[1][0].imshow(image[:250, :250, :])
ax[1][0].set_title('Original')
ax[1][1].imshow(img_hema[:250, :250], cmap=plt.cm.gray)
ax[1][1].set_title('Hematoxylin')
ax[1][2].imshow(img_eosin[:250, :250], cmap=plt.cm.gray)
ax[1][2].set_title('Eosin')
ax[1][3].imshow(img_dab[:250, :250], cmap=plt.cm.gray)
ax[1][3].set_title('DAB')

for a in ax:
  for b in a:
    b.axis('off')
    
fig.tight_layout()

## Gaussian Blurring

Gaussian blurring simply blurs the image by "blending" or averaging nearby pixel values. This is used as a smoothing operation to get rid of local specks in the image. You can control the "amount" of blending by setting the standard deviation value of the Gaussian used to filter the image.

In [ ]:
from skimage import img_as_float
from scipy.ndimage import gaussian_filter
from skimage.morphology import reconstruction

# Convert image to a float for subtraction from the original
img_nuc = img_as_float(img_hema)

# Run a simple gaussian filter to blur the image
img_nuc = gaussian_filter(img_nuc, 1)

In [ ]:
# Display
fig, ax = plt.subplots(2, 2, figsize=(15,10))

ax[0][0].imshow(img_hema, cmap=plt.cm.gray)
ax[0][0].set_title('Hematoxylin Channel')
ax[0][1].imshow(img_nuc, cmap=plt.cm.gray)
ax[0][1].set_title('Gaussian Filtered Image')

ax[1][0].imshow(img_hema[:250, :250], cmap=plt.cm.gray)
ax[1][0].set_title('Hematoxylin Channel')
ax[1][1].imshow(img_nuc[:250, :250], cmap=plt.cm.gray)
ax[1][1].set_title('Gaussian Filtered Image')

for a in ax:
  for b in a:
    b.axis('off')
    
fig.tight_layout()

## Image Reconstruction through Dilation

In this process, we create a map of low-intensity regions of the image (those below a threshold) and subtrac them from the image, so that the only image areas that remain are bright spots.

In [ ]:
# Create a reconstruction of the image where low-intensity 
# regions in a neighborhood are suppressed

# First create a "seed": a matrix with the minimum value of the image
seed = img_nuc - 0.125

# Next create a "mask": Just the image itself
mask = img_nuc

# Create a "dilated" image: reconstruction through dilation
dilated = reconstruction(seed, mask, method='dilation')

img_nuc_filtered = img_nuc - dilated

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(20, 10))

ax[0][0].imshow(img_nuc, cmap='gray')
ax[0][0].set_title('original image')
ax[0][1].imshow(dilated, vmin=img_nuc.min(), vmax=img_nuc.max(), cmap='gray')
ax[0][1].set_title('dilated')
ax[0][2].imshow(img_nuc_filtered, cmap='gray')
ax[0][2].set_title('image - dilated')

ax[1][0].imshow(img_nuc[:250, :250], cmap='gray')
ax[1][0].set_title('original image')
ax[1][1].imshow(dilated[:250, :250], vmin=img_nuc.min(), vmax=img_nuc.max(), cmap='gray')
ax[1][1].set_title('dilated')
ax[1][2].imshow(img_nuc_filtered[:250, :250], cmap='gray')
ax[1][2].set_title('image - dilated')

for a in ax:
  for b in a:
    b.axis('off')

plt.tight_layout()

## Image Thresholding

Our reconstructed image is now suitable for thresholding -- but what value should we use to threshold?

We can take a look at the image histogram to give us an idea of how graylevel values are distributed across the image.

In [ ]:
# Plot a histogram of an image
f, ax = plt.subplots(2,1,figsize=(10,6))

ax[0].hist(np.ravel(img_nuc_filtered), bins=256, density=True)
ax[0].set(xlabel="Image Intensity",
       ylabel="Density",
       title="Image Histogram")

ax[1].hist(np.ravel(img_nuc_filtered), bins=256, density=True)
ax[1].set(xlabel="Image Intensity",
       ylabel="Density",
       title="Image Histogram",
       ylim=[0, 40])

for a in ax:
  a.grid(linestyle=':')
plt.tight_layout()
plt.show()

Imagine drawing a vertical line somewhere on the X-axis, where everything below that line (the darker areas) becomes black, and everything above the line (the lighter areas) becomes white. What number should we choose to get the optimal segmentation?

Thankfully, there is a simple algorithm for greylevel images called [Otsu's Threshold](https://en.wikipedia.org/wiki/Otsu's_method), a very nice explanation of which can be found [here](http://www.labbookpages.co.uk/software/imgProc/otsuThreshold.html).

In [ ]:
img_otsu = img_nuc_filtered > filters.threshold_otsu(img_nuc_filtered)

print('Calculated Otsu Threshold: {:.4f}'.format(filters.threshold_otsu(img_nuc_filtered)))

In [ ]:
# Display
fig, ax = plt.subplots(2, 2, figsize=(12, 12))

ax[0][0].imshow(img_nuc_filtered, cmap='gray')
ax[0][0].set_title('filtered image')
ax[0][1].imshow(img_otsu, cmap='gray')
ax[0][1].set_title('otsu thresholded image')

ax[1][0].imshow(img_nuc_filtered[:250, :250], cmap='gray')
ax[1][0].set_title('filtered image')
ax[1][1].imshow(img_otsu[:250, :250], cmap='gray')
ax[1][1].set_title('otsu thresholded image')

for a in ax:
  for b in a:
    b.axis('off')

fig.tight_layout()

## Image Cleanup: Removing Small Specks and Holes

Otsu rarely gives us a "clean" image, so we need to perform some operations to fix what we've got. Two quick ones are removing small objects (aka "Area Threshold") and filling in small holes. 

In [ ]:
from skimage.morphology import remove_small_objects, remove_small_holes

img_open = remove_small_objects(img_otsu, min_size=64)
img_close = remove_small_holes(img_open, area_threshold=64)

# Replace img_nuc_bin with the final step of processing for the next section
img_nuc_bin = img_close

In [ ]:
# Display
fig, ax = plt.subplots(2, 2, figsize=(12, 12))

ax[0][0].imshow(img_otsu, cmap='gray')
ax[0][0].set_title('Before Cleaning')
ax[0][1].imshow(img_close, cmap='gray')
ax[0][1].set_title('After Cleaning')

ax[1][0].imshow(img_otsu[:250, :250], cmap='gray')
ax[1][0].set_title('Before Cleaning')
ax[1][1].imshow(img_close[:250, :250], cmap='gray')
ax[1][1].set_title('After Cleaning')

for a in ax:
  for b in a:
    b.axis('off')

fig.tight_layout()

## Watershed Transform

Now that we have our cleaned image, we **almost** have our nuclei. However, there's a small problem: Nuclei that are touching are currently treated as one object. Is it possible to split apart nuclei?

This is a difficult problem, but one easy approach is the [watershed transform](https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_watershed.html), which is also nicely described in the Mathworks MATLAB documentation [here](https://www.mathworks.com/help/images/ref/watershed.html). This approach is useful when you have round objects in binary images and you want to separate them out where they touch. 

In [ ]:
# Additional imports
from scipy.ndimage import distance_transform_edt
from scipy.ndimage import generate_binary_structure, grey_dilation
from skimage.morphology import watershed, label
from skimage.feature import peak_local_max
from skimage.color import label2rgb
from skimage.segmentation import clear_border

In [ ]:
# Get the euclidean distance transform -- distance from each object-pixel to the background
img_distance = -distance_transform_edt(img_nuc_bin)

# Set the background to a very negative number
img_distance[~img_nuc_bin] = -100

In [ ]:
# Plot the distance map
fig, ax = plt.subplots(2,1,figsize=(20,10))

ax[0].imshow(img_distance, cmap=plt.cm.gray)
ax[1].imshow(img_distance[:250, :250], cmap=plt.cm.gray)

for a in ax:
  a.axis('off')
fig.tight_layout()

This complex bit of code simply prepares the distance image by suppressing local minima, which often leads to over-segmentation in watershed. Details [here](https://github.com/janelia-flyem/gala/blob/master/gala/morpho.py).

In [ ]:
# Suppress local minima in the image to prevent over-segmentation
# See: https://github.com/janelia-flyem/gala/blob/master/gala/morpho.py

# The height threshold is determined empirically, based on distances of the objects in the image
hthreshold = 1

# Invert the distance image by subtracting it from the maximum value in the image 
maxval = img_distance.max()
img_inv = maxval - img_distance.astype(float)

# The marker 
marker = img_inv - hthreshold

mask = img_inv

sel = generate_binary_structure(marker.ndim, 1)
diff = True
while diff:
    markernew = grey_dilation(marker, footprint=sel)
    markernew = np.minimum(markernew, mask)
    diff = (markernew - marker).max() > 0
    marker = markernew

filled = maxval - marker

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(20,10))

ax[0].imshow(filled, cmap=plt.cm.gray)
ax[1].imshow(filled[:250, :250], cmap=plt.cm.gray)

for a in ax:
  a.axis('off')
fig.tight_layout()

In [ ]:
# Perform watershed
labels_ws = watershed(filled, mask=img_nuc_bin, watershed_line=True)

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(20,10))

ax[0][0].imshow(image)
ax[0][0].set_title('Original Image')
ax[0][1].imshow(img_nuc_bin, cmap='gray')
ax[0][1].set_title('Binary Nuclear Image')
ax[0][2].imshow(label2rgb(labels_ws, bg_label=0))
ax[0][2].set_title('Watershed Segmentation')

ax[1][0].imshow(image[:250, :250, :])
ax[1][0].set_title('Original Image')
ax[1][1].imshow(img_nuc_bin[:250, :250], cmap='gray')
ax[1][1].set_title('Binary Nuclear Image')
ax[1][2].imshow(label2rgb(labels_ws[:250, :250], bg_label=0))
ax[1][2].set_title('Watershed Segmentation')

for a in ax:
  for b in a:
    b.axis('off')
    
fig.tight_layout()

## EXPERIMENTAL: DO NOT RUN

Gabor Texture Analysis

In [ ]:
from scipy import ndimage as ndi

def power(image, kernel):
    # Normalize images for better comparison.
    image = (image - image.mean()) / image.std()
    return np.sqrt(ndi.convolve(image, np.real(kernel), mode='wrap')**2 +
                   ndi.convolve(image, np.imag(kernel), mode='wrap')**2)

In [ ]:
from skimage.filters import gabor_kernel

# Plot a selection of the filter bank kernels and their responses.
results = []
kernel_params = []
for theta in (0, 1):
    theta = theta / 4. * np.pi
    for frequency in (0.1, 0.4):
        kernel = gabor_kernel(frequency, theta=theta)
        params = 'theta=%d,\nfrequency=%.2f' % (theta * 180 / np.pi, frequency)
        kernel_params.append(params)

        #print(np.shape(kernel))

        # Save kernel and the power image for each image
        results.append((kernel, [power(img, kernel) for img in images]))

In [ ]:
theta = 0
frequency=0.1
kernel = gabor_kernel(frequency=frequency, theta=theta)
result = np.sqrt(ndi.convolve(images[0], np.real(kernel), mode='wrap')**2 + ndi.convolve(images[0], np.imag(kernel), mode='wrap')**2, dtype=np.float)

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(10,6))

axes[0].imshow(result, cmap=plt.cm.gray)
axes[0].set_title('Result of Gabor Filtering')
axes[1].imshow(np.real(kernel), cmap=plt.cm.gray)
axes[1].set_title('Gabor Kernel')

for ax in axes:
  ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(5, 6))
plt.gray()

for label, (kernel, powers), ax_row in zip(kernel_params, results, axes[1:]):
    # Plot Gabor kernel
    ax = ax_row[0]
    ax.imshow(np.real(kernel))
    ax.set_ylabel(label, fontsize=7)
    ax.set_xticks([])
    ax.set_yticks([])

    # Plot Gabor responses with the contrast normalized for each filter
    vmin = np.min(powers)
    vmax = np.max(powers)
    for patch, ax in zip(powers, ax_row[1:]):
        ax.imshow(patch, vmin=vmin, vmax=vmax)
        ax.axis('off')

plt.show()